In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Paraíba - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [7]:
data = pd.read_csv('2003_01_model_input_PB.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego,Paraíba - Consumo de Cimento (t)
0,2003,0.656510,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,1475.829837,2.441479,3.017292e+07,1.631334e+06,7.291169,2.736342e+07,8.315958,392.755000
1,2004,0.661779,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,1587.871571,3.057163,3.061905e+07,1.650726e+06,7.318404,2.750422e+07,8.243195,354.563000
2,2005,0.666544,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,1686.293242,3.724982,3.105294e+07,1.663425e+06,7.349948,2.765335e+07,8.219443,365.065000
3,2006,0.670820,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,1796.153586,4.383607,3.145782e+07,1.689128e+06,7.378438,2.778800e+07,8.131041,430.470000
4,2007,0.674620,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,1888.085455,4.964201,3.179414e+07,1.705364e+06,7.389855,2.785711e+07,8.106365,523.308000
5,2008,0.677959,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,1945.158359,5.342951,3.208948e+07,1.723657e+06,7.399995,2.790503e+07,8.076245,707.783000
6,2009,0.680849,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,1991.871625,5.534727,3.229855e+07,1.737699e+06,7.397195,2.790008e+07,8.105416,734.672000
7,2010,0.683306,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,2087.894628,5.851130,3.308896e+07,1.751827e+06,7.427988,2.807403e+07,8.069181,834.169000
8,2011,0.685344,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,2212.198689,6.127922,3.384049e+07,1.755792e+06,7.458472,2.823647e+07,8.055989,946.248000
9,2012,0.686846,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,2280.624032,6.316335,3.453206e+07,1.759354e+06,7.482244,2.837952e+07,10.160575,1103.437000


In [8]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego
0,0.656510,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,1475.829837,2.441479,3.017292e+07,1.631334e+06,7.291169,2.736342e+07,8.315958
1,0.661779,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,1587.871571,3.057163,3.061905e+07,1.650726e+06,7.318404,2.750422e+07,8.243195
2,0.666544,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,1686.293242,3.724982,3.105294e+07,1.663425e+06,7.349948,2.765335e+07,8.219443
3,0.670820,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,1796.153586,4.383607,3.145782e+07,1.689128e+06,7.378438,2.778800e+07,8.131041
4,0.674620,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,1888.085455,4.964201,3.179414e+07,1.705364e+06,7.389855,2.785711e+07,8.106365
5,0.677959,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,1945.158359,5.342951,3.208948e+07,1.723657e+06,7.399995,2.790503e+07,8.076245
6,0.680849,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,1991.871625,5.534727,3.229855e+07,1.737699e+06,7.397195,2.790008e+07,8.105416
7,0.683306,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,2087.894628,5.851130,3.308896e+07,1.751827e+06,7.427988,2.807403e+07,8.069181
8,0.685344,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,2212.198689,6.127922,3.384049e+07,1.755792e+06,7.458472,2.823647e+07,8.055989
9,0.686846,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,2280.624032,6.316335,3.453206e+07,1.759354e+06,7.482244,2.837952e+07,10.160575


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      354.563000
1      365.065000
2      430.470000
3      523.308000
4      707.783000
5      734.672000
6      834.169000
7      946.248000
8     1103.437000
9     1223.444000
10    1240.232857
11    1076.248286
12     928.236857
13     786.338000
14     707.565000
15     769.080000
16    1051.694000
17    1112.087000
18    1064.438000
19            NaN
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego
0,-2.021610,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.763177,-2.288696,-1.647449,-2.273161,-2.121047,-2.002019,-0.806111
1,-1.618397,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.454969,-1.839656,-1.433705,-1.734803,-1.707018,-1.626108,-0.858247
2,-1.253728,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.184227,-1.352592,-1.225822,-1.382234,-1.227486,-1.227951,-0.875266
3,-0.926533,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.882019,-0.872234,-1.031838,-0.668651,-0.794371,-0.868441,-0.938608
4,-0.635739,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.629130,-0.448787,-0.870702,-0.217902,-0.620818,-0.683925,-0.956288
5,-0.380275,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.472131,-0.172551,-0.729197,0.289934,-0.466668,-0.555978,-0.977870
6,-0.159069,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.343631,-0.032682,-0.629030,0.679792,-0.509224,-0.569210,-0.956968
7,0.028948,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.079487,0.198081,-0.250333,1.072005,-0.041110,-0.104766,-0.982932
8,0.184850,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.262453,0.399956,0.109738,1.182094,0.422316,0.328924,-0.992384
9,0.299823,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.450680,0.537372,0.441076,1.280980,0.783687,0.710858,0.515590


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      354.563000
1      365.065000
2      430.470000
3      523.308000
4      707.783000
5      734.672000
6      834.169000
7      946.248000
8     1103.437000
9     1223.444000
10    1240.232857
11    1076.248286
12     928.236857
13     786.338000
14     707.565000
15     769.080000
16    1051.694000
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego
17,0.98374,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,2.45631,1.406933,0.77241,-0.009612,0.301112,0.401403,1.165086


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    1112.087
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [14]:
train_input.iloc[-(len(train_input)//5):]

,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego
14,1.345395,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,0.651544,0.836267,1.188614,-0.094510,0.908155,0.969461,1.175737
15,1.230676,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,1.322697,0.851838,1.134725,-0.144853,0.795288,0.824407,1.197052
16,1.147268,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,2.374644,1.141098,1.020077,-0.057621,0.603047,0.668766,1.254680


In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3816280310, 529416774, 233115314, 818413576, 3700500743, 1367250547, 375527965, 401340216, 2324246653, 2526101199, 49223514, 2472208144, 1808731739, 1675786817, 3776410460, 1423308009, 744656852, 4106379027, 1718910941, 1574682417, 1457498004, 1013990957, 1355655455, 4036598963, 4183129626, 916855918, 2717764234, 230084442, 783615584, 3443357221, 2312703914, 3254703386, 2387136448, 139079437, 2333889099, 2033019636, 637090212, 3672603153, 2060521890, 3235009535, 1154989708, 3546047291, 434532367, 2208296873, 2039189099, 956064465, 261193965, 1805757665, 3718189497, 1047177251, 573723196, 3547880240, 3421313860, 799379391, 2749152515, 1125716075, 1567182628, 12144197, 3950656642, 213433170, 588054878, 2133820704, 276342223, 968263545, 3296801979, 2817539864, 3134521397, 1036332865, 4200232509, 2844448833, 2112779167, 3366080893, 1614616218, 1786216166, 2406250380, 3043407085, 2423468648, 3682573772, 4028870891, 749383246, 1324149916, 2498023633, 266153091, 2886685564, 2145854832, 15918

val_loss: 25975.564453125


Step: 88 ___________________________________________
val_loss: 12950.671875


Step: 89 ___________________________________________
val_loss: 34853.15234375


Step: 90 ___________________________________________
val_loss: 22515.4375


Step: 91 ___________________________________________
val_loss: 40848.90625


Step: 92 ___________________________________________
val_loss: 22946.693359375


Step: 93 ___________________________________________
val_loss: 22473.8046875


Step: 94 ___________________________________________
val_loss: 25874.197265625


Step: 95 ___________________________________________
val_loss: 26042.8125


Step: 96 ___________________________________________
val_loss: 28220.447265625


Step: 97 ___________________________________________
val_loss: 27852.7734375


Step: 98 ___________________________________________
val_loss: 25694.404296875


Step: 99 ___________________________________________
val_loss: 22470.9921875


final_seed: 3700500743


In [19]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 438ms/step - loss: 731751.0000 - val_loss: 702163.4375
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 709184.4375 - val_loss: 345027.7812
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 452832.5625 - val_loss: 781319.6875
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 771906.5000 - val_loss: 238064.7344
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 380432.1875 - val_loss: 61926.8438
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 191698.3906 - val_loss: 279319.3438
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 176095.4688 - val_loss: 80434.7578
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 456747.5625 - val_loss: 926006.8125
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 641133.3125 - val_loss: 200411.5000
Epoch 10/10

1/1 [==============================] - 0s 14ms/step - loss: 113567.8203 - val_loss: 39756.2969
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 145176.5938 - val_loss: 54272.8789
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 135205.2188 - val_loss: 45896.7305
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 112058.6016 - val_loss: 38617.7031
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 127075.3672 - val_loss: 151206.9531
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 167675.9688 - val_loss: 47357.5195
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 113471.8359 - val_loss: 40651.7383
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 134942.7969 - val_loss: 68753.1016
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 125046.8828 - val_loss: 38977.7930
Epoch 84/10000
1/1 [====

1/1 [==============================] - 0s 14ms/step - loss: 57913.7930 - val_loss: 23473.1094
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 74323.3672 - val_loss: 29097.2090
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 58412.7656 - val_loss: 23287.3691
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 56133.7383 - val_loss: 23334.2324
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 57285.0352 - val_loss: 23237.4355
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 64002.2656 - val_loss: 25855.8105
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 70963.1484 - val_loss: 26326.1348
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 56238.9570 - val_loss: 22534.6621
Epoch 157/10000
1/1 [==============================] - 0s 13ms/step - loss: 54193.0703 - val_loss: 23141.5703
Epoch 158/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 45188.9062 - val_loss: 25234.2949
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 36522.4453 - val_loss: 23340.5488
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 39634.5977 - val_loss: 22855.3848
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 33785.3633 - val_loss: 24154.4004
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 34489.4336 - val_loss: 22516.2812
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 36394.8242 - val_loss: 22932.2246
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 36841.2930 - val_loss: 22533.5723
Epoch 230/10000
1/1 [==============================] - 0s 13ms/step - loss: 35260.1875 - val_loss: 22610.3145
Epoch 231/10000
1/1 [==============================] - 0s 13ms/step - loss: 33472.3789 - val_loss: 26393.1973
Epoch 232/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 23867.9570 - val_loss: 26287.9355
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 28451.0215 - val_loss: 24289.1465
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 26328.5430 - val_loss: 27445.7812
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 24582.2148 - val_loss: 25954.8496
Epoch 301/10000
1/1 [==============================] - 0s 13ms/step - loss: 24811.8594 - val_loss: 28529.1953
Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 23642.1270 - val_loss: 26393.8281
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 27521.1953 - val_loss: 24752.5215
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 23191.6660 - val_loss: 26597.7559
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 22260.5488 - val_loss: 28366.0840
Epoch 306/10000
1/1 [=====

...

1/1 [==============================] - 0s 14ms/step - loss: 19960.4277 - val_loss: 46190.4102
Epoch 964/10000
1/1 [==============================] - 0s 15ms/step - loss: 20966.9062 - val_loss: 43915.0273
Epoch 965/10000
1/1 [==============================] - 0s 13ms/step - loss: 22144.1582 - val_loss: 44208.0664
Epoch 966/10000
1/1 [==============================] - 0s 13ms/step - loss: 19893.5781 - val_loss: 46050.5430
Epoch 967/10000
1/1 [==============================] - 0s 14ms/step - loss: 19944.4766 - val_loss: 45343.2461
Epoch 968/10000
1/1 [==============================] - 0s 14ms/step - loss: 19791.3887 - val_loss: 45858.3164
Epoch 969/10000
1/1 [==============================] - 0s 13ms/step - loss: 19832.1055 - val_loss: 37360.1602
Epoch 970/10000
1/1 [==============================] - 0s 14ms/step - loss: 19441.1133 - val_loss: 46579.6133
Epoch 971/10000
1/1 [==============================] - 0s 13ms/step - loss: 20356.2148 - val_loss: 44434.0156
Epoch 972/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 23346.7344 - val_loss: 36690.5820
Epoch 1038/10000
1/1 [==============================] - 0s 13ms/step - loss: 23609.5273 - val_loss: 36256.3438
Epoch 1039/10000
1/1 [==============================] - 0s 13ms/step - loss: 22214.5742 - val_loss: 43499.4141
Epoch 1040/10000
1/1 [==============================] - 0s 13ms/step - loss: 22261.9629 - val_loss: 37824.4961
Epoch 1041/10000
1/1 [==============================] - 0s 13ms/step - loss: 21956.9414 - val_loss: 38948.7461
Epoch 1042/10000
1/1 [==============================] - 0s 13ms/step - loss: 24046.1191 - val_loss: 36707.9531
Epoch 1043/10000
1/1 [==============================] - 0s 13ms/step - loss: 21253.6641 - val_loss: 39263.2930
Epoch 1044/10000
1/1 [==============================] - 0s 13ms/step - loss: 22761.4883 - val_loss: 36539.2461
Epoch 1045/10000
1/1 [==============================] - 0s 13ms/step - loss: 23526.2871 - val_loss: 37829.9102
Epoch 1046/10000
1

Epoch 1111/10000
1/1 [==============================] - 0s 13ms/step - loss: 20011.1406 - val_loss: 41159.7148
Epoch 1112/10000
1/1 [==============================] - 0s 13ms/step - loss: 19975.0195 - val_loss: 42293.1953
Epoch 1113/10000
1/1 [==============================] - 0s 13ms/step - loss: 20521.1289 - val_loss: 40291.5000
Epoch 1114/10000
1/1 [==============================] - 0s 14ms/step - loss: 20431.2852 - val_loss: 40818.8398
Epoch 1115/10000
1/1 [==============================] - 0s 13ms/step - loss: 24484.8340 - val_loss: 35237.7305
Epoch 1116/10000
1/1 [==============================] - 0s 13ms/step - loss: 20951.3906 - val_loss: 41363.0859
Epoch 1117/10000
1/1 [==============================] - 0s 14ms/step - loss: 23677.8848 - val_loss: 37387.5977
Epoch 1118/10000
1/1 [==============================] - 0s 13ms/step - loss: 20534.8164 - val_loss: 41076.0586
Epoch 1119/10000
1/1 [==============================] - 0s 13ms/step - loss: 20278.5840 - val_loss: 41602.6406
E

Epoch 1185/10000
1/1 [==============================] - 0s 13ms/step - loss: 19330.8848 - val_loss: 45343.6875
Epoch 1186/10000
1/1 [==============================] - 0s 14ms/step - loss: 21719.9531 - val_loss: 39585.2812
Epoch 1187/10000
1/1 [==============================] - 0s 13ms/step - loss: 20072.5156 - val_loss: 45321.4727
Epoch 1188/10000
1/1 [==============================] - 0s 13ms/step - loss: 19552.7344 - val_loss: 42807.3047
Epoch 1189/10000
1/1 [==============================] - 0s 13ms/step - loss: 19777.2441 - val_loss: 44484.3242
Epoch 1190/10000
1/1 [==============================] - 0s 14ms/step - loss: 19917.9844 - val_loss: 39108.9258
Epoch 1191/10000
1/1 [==============================] - 0s 13ms/step - loss: 19914.2500 - val_loss: 42762.5195
Epoch 1192/10000
1/1 [==============================] - 0s 14ms/step - loss: 19717.5781 - val_loss: 42995.5273
Epoch 1193/10000
1/1 [==============================] - 0s 13ms/step - loss: 19977.2539 - val_loss: 43035.1758
E

Epoch 1259/10000
1/1 [==============================] - 0s 13ms/step - loss: 19281.7363 - val_loss: 42513.7031
Epoch 1260/10000
1/1 [==============================] - 0s 14ms/step - loss: 19203.9922 - val_loss: 42203.9531
Epoch 1261/10000
1/1 [==============================] - 0s 14ms/step - loss: 19699.3281 - val_loss: 40939.3867
Epoch 1262/10000
1/1 [==============================] - 0s 13ms/step - loss: 19014.1387 - val_loss: 41641.4961
Epoch 1263/10000
1/1 [==============================] - 0s 13ms/step - loss: 19234.2793 - val_loss: 42936.8008
Epoch 1264/10000
1/1 [==============================] - 0s 13ms/step - loss: 19351.6484 - val_loss: 41215.1875
Epoch 1265/10000
1/1 [==============================] - 0s 14ms/step - loss: 19487.1816 - val_loss: 41397.3125
Epoch 1266/10000
1/1 [==============================] - 0s 13ms/step - loss: 19176.4297 - val_loss: 42437.0430
Epoch 1267/10000
1/1 [==============================] - 0s 13ms/step - loss: 19257.7227 - val_loss: 40098.9727
E

In [20]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[1112.087](test_target) - [[988.84906]](prediction) = 123.23793994140624


In [21]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [22]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego
0,-1.704907,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.681319,-1.704754,-1.577007,-1.556934,-1.816118,-1.791670,1.871815
1,-1.107554,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.113171,-1.172252,-1.074050,-1.076262,-1.181625,-1.148529,1.007625
2,-0.567302,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.614089,-0.594658,-0.584887,-0.761471,-0.446748,-0.467330,0.725527
3,-0.082567,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.057003,-0.025015,-0.128428,-0.124350,0.216995,0.147749,-0.324413
4,0.348240,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.409171,0.477138,0.250739,0.278100,0.482963,0.463435,-0.617481
5,0.726707,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.698580,0.804717,0.583710,0.731521,0.719195,0.682338,-0.975215
6,1.054419,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.935456,0.970584,0.819410,1.079605,0.653979,0.659698,-0.628747
7,1.332964,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.422375,1.244240,1.710514,1.429792,1.371358,1.454309,-1.059112


train_target:


,Paraíba - Consumo de Cimento (t)
0,354.563
1,365.065
2,430.470
3,523.308
4,707.783
5,734.672
6,834.169
7,946.248


test_input:


...

,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego
17,0.98374,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,2.45631,1.406933,0.77241,-0.009612,0.301112,0.401403,1.165086


test_target:


,Paraíba - Consumo de Cimento (t)
17,1112.087


1/1 [==============================] - 0s 34ms/step
Error: 123.23793994140624




[806.1519775390625,
 986.1134033203125,
 1155.5794677734375,
 1153.156982421875,
 1215.7506103515625,
 1005.986083984375,
 992.68017578125,
 910.349609375,
 704.8887939453125,
 988.8490600585938]

In [23]:
display(targets)
display(predictions)

[1103.437,
 1223.444,
 1240.2328571428573,
 1076.248285714286,
 928.2368571428572,
 786.338,
 707.565,
 769.08,
 1051.694,
 1112.087]

[806.1519775390625,
 986.1134033203125,
 1155.5794677734375,
 1153.156982421875,
 1215.7506103515625,
 1005.986083984375,
 992.68017578125,
 910.349609375,
 704.8887939453125,
 988.8490600585938]

In [24]:
mae = mean_absolute_error(predictions, targets)
mae

209.97674735630576

In [25]:
porcentage = mae/np.mean(targets)
porcentage

0.21001112617766107